
# From First Flush to Final Flow: A Stormwater Model in Python

*Product Rule in the Real World: A Stormwater Modeling Case Study*

---

## ⚡️ Why Stormwater Modeling Matters

When a storm hits, it's not just rain that flows through our cities — it's a cascade of unseen consequences. Water moves swiftly over impervious surfaces like streets, rooftops, and parking lots, unable to soak into the ground. As it travels, this runoff collects a mix of contaminants: oil and grease from roadways, fertilizers and pesticides from lawns, heavy metals from vehicles, and even pet waste and trash. All of it ends up in our storm drains, eventually discharging into rivers, lakes, and coastal waters — untreated.

This surge of polluted water can overwhelm ecosystems, degrade water quality, and pose risks to both public health and wildlife. It's not just an environmental issue — it's an engineering one, too.

As civil engineers, we are often called upon to quantify this impact. How much pollutant is entering a watershed during a given storm? When does the peak loading occur? How long does it persist? These are vital questions when designing stormwater infrastructure, modeling water quality, or ensuring regulatory compliance under programs like the Clean Water Act's MS4 permit system.

But answering those questions accurately isn’t trivial. Storm events are dynamic. Flow changes over time. Pollutant concentrations vary — often spiking during the initial flush, then tapering off. Relying on static assumptions or overly simplistic math leads to models that might be technically correct on paper but totally disconnected from how water and pollutants behave in the real world.

This is where Python enters the scene.

As developers and data modelers, we want to build tools that help us simulate and understand these processes. And we want those tools to reflect both mathematical integrity and physical reality. Python gives us that power: symbolic math with libraries like SymPy, numerical evaluation with NumPy and SciPy, and visualization with Matplotlib or Plotly.

In this post, we’ll build a pollutograph model — a time-based representation of pollutant load — using the product rule from calculus. We’ll walk through an initial version, catch a modeling flaw, and improve it using a better representation of concentration over time. Along the way, you’ll see how code, calculus, and engineering intuition come together to tell the story of a storm.

---

## 📊 The Basic Model: Product Rule in Action

Pollutant loading during a storm can be expressed as:

$$
L(t) = Q(t) \cdot C(t)
$$

Where:
- `L(t)`: pollutant load at time `t`
- `Q(t)`: flow rate
- `C(t)`: concentration of pollutants in runoff

Let’s start with simple assumptions:
- `Q(t) = 3t` (flow increases linearly)
- `C(t) = 10 - 0.2t` (concentration decreases linearly)

Using the **product rule**, the rate of change of pollutant load is:

$$
L'(t) = Q'(t)C(t) + Q(t)C'(t)
$$

Symbolically:
- `Q'(t) = 3`
- `C'(t) = -0.2`
- So, `L'(t) = 3(10 - 0.2t) + 3t(-0.2) = 30 - 1.2t`

---

## ⚠️ The Graph That Didn’t Make Sense

Plotting L'(t) = 30 - 1.2t — the rate of change of pollutant load — produced a straight, declining line. Mathematically, it looked fine: the derivative was correct, the calculus checked out, and the function behaved as expected for the chosen equations.

But something felt... off.

In the real world, stormwater systems don't behave that cleanly. The graph suggested that pollutant loading was highest at the very start of the storm and then steadily decreased — implying that maximum contamination occurred the moment rainfall began. And that’s not how storms work.

Let’s break it down:

Pollutant loading doesn't peak at time zero. In fact, when the rain first hits the ground, the flow rate is still ramping up. Pipes and culverts haven’t yet filled. The watershed hasn’t responded. There's water movement, but not yet at full volume.

First flush matters. This is a well-known phenomenon in stormwater engineering: the initial runoff carries a disproportionately high concentration of pollutants. Dirt, oil, and debris accumulate between storms, and the early rain flushes them out quickly. So yes — concentrations are high at the start, but flow is still low, meaning loading hasn't peaked yet.

Load is a function of both flow and concentration. Early in the storm, we see high concentration but modest flow. Later, flow increases while concentration decreases. Somewhere in between, the two balance to produce a peak in total loading — the moment when the mass of pollutants being delivered per unit time hits a maximum.

So, the assumption that concentration drops linearly with time (C(t) = 10 - 0.2t) oversimplified the system. It created a model where the early high concentration dominated the equation, making it seem like maximum loading happened before the storm even had time to build up. That assumption failed to capture the dynamic nature of storm response — the rise and fall of both flow and pollutant concentration.

In other words: the math was right, but the model was wrong.

That’s the key lesson here. Modeling isn’t just about getting the calculus correct — it’s about making sure the underlying assumptions reflect the physical world. When they don’t, even a “correct” equation can lead you astray.

---

## ↺ Reworking the Model: Exponential Decay

To better reflect reality, we updated the concentration function to use **exponential decay**:

$$
C(t) = 10e^{-0.1t}
$$

This change wasn’t arbitrary — it was rooted in observation and experience. In urban hydrology, pollutant concentrations typically start high during the initial flush, as accumulated contaminants are swept into the flow. But as time passes and the “available” pollutants are washed off, the concentration tends to drop off rapidly, not gradually. That decay often resembles an exponential curve more than a straight line.

So rather than decreasing at a constant rate, this new model says:
- Concentration is high early on
- Drops quickly as the most accessible pollutants are flushed
- Tapers off more slowly as fewer pollutants remain

We kept the original flow parameter as `Q(t) = 3t`

Flow increases linearly as the storm progresses — a simplification, but a decent approximation for a steadily intensifying storm with no sudden runoff interruptions.

The new pollutant load model becomes:

$$
L(t) = 3t \cdot 10e^{-0.1t} = 30t \cdot e^{-0.1t}
$$

We used SymPy to symbolically differentiate this expression and obtain the derivative,

`L′(t)`, which represents the rate of change of pollutant loading over time.

When we plotted the new function, the shape was dramatically different from our initial attempt:
- ✅ An initial increase in load — as both flow and concentration are still relatively high
- ✅ A clearly visible peak — the point of maximum pollutant transport
- ✅ A smooth, natural decline — reflecting increasing dilution and runoff "cleansing" over time

This is exactly the kind of curve you'd expect from a real pollutograph — the time-based profile of pollutant load during a storm. It behaves like a classic rise-and-fall scenario:
- A steep increase as the storm ramps up
- A short-lived peak as the system hits its most intense loading
- A decline as the runoff volume overtakes remaining concentration

What’s powerful here is how a small change in assumptions (switching from linear to exponential decay) transforms the model from a rigid abstraction into something that mirrors the physical world. This is where modeling becomes meaningful — not just solving for the right equation, but choosing the right form for the problem.

This aligns beautifully with what we expect from a real storm event.

> [stormwater loading graph](../figures/graphs/stormwater_loading.png)

---

## 🔬 What Happens When the Decay Rate Changes?
Once we adopted exponential decay for concentration, we had a much more realistic model. But exponential functions are flexible — and that gives us an opportunity to explore how different decay rates affect pollutant loading during a storm.

Let’s generalize the concentration function:

$$
C(t) = 10 \cdot e^{−k \dot t}
$$

Where:
- k is the decay constant
- Larger values of `k` mean faster decay (concentration drops off quickly)
- Smaller values of `k` mean slower decay (pollutants persist longer)
Then the load function becomes:
$$
L(t) = Q(t) \cdot C(t) = 3t \cdot 10e^{−kt} = 30t \cdot e^{−kt}
$$

- To understand how `k` affects pollutant behavior, let’s look at what changes when we vary `k`:

#### 📈 Visual Comparison of Different Decay Rates
> [decay rates table](../figures/tables/decay_rates.png)



#### 🧠 Interpretation:
Small `k` values (slow decay):
- Pollutants remain in the runoff longer, so the loading continues to rise even as flow increases. The result is a wider peak and longer pollutograph.
- Large `k` values (fast decay):
Pollutants wash off quickly. Even though flow might still be rising, the concentration has already dropped — leading to a shorter, sharper peak.

#### 🔍 Real-World Implications:
Changing `k` simulates different types of urban environments or land use conditions:

- A heavily industrialized area might have more surface contaminants, leading to a slower decay (lower `k`).
- A residential zone with frequent rainfall might have fewer accumulated pollutants, causing quicker flushing (higher `k`).
- By adjusting `k`, you're not just tweaking a formula — you’re simulating different physical scenarios.

---



## 📉 Key Takeaways

- The product rule is powerful—but it's only as good as your assumptions.
- A mathematically correct model can still be **physically wrong**.
- Simple updates (like switching from linear to exponential decay) can greatly improve realism.
- QA/QC isn't just for infrastructure drawings—it's essential in modeling too.

---

## 🔗 Bonus: Revenue Modeling with the Product Rule

Want to see how this exact same math applies to **business and finance**?

I also modeled **revenue optimization** using the product rule, exploring how pricing strategies affect units sold and total profit.

Check out the example in my GitHub repo:
> [💻 Product Rule — Revenue Optimization in Python](https://github.com/Pencils-and-Python/Pencils-Python-Derivatives)

---

## 🏆 Closing Thoughts

This project reminded me that modeling is just as much about critical thinking as it is about calculus or code. Writing a correct equation is one thing — but writing a useful one? That’s something else entirely.

You can follow all the mathematical rules, write syntactically perfect code, and still end up with a model that doesn’t make sense. Why? Because math doesn't care about context — you do. And in engineering, context is everything.

Whether you’re tracking pollution in a watershed or optimizing profit in a pricing model, you’re making assumptions. About flow. About behavior. About time, space, systems, and people. And if those assumptions are off — even by a little — your beautifully constructed model might lead you to the wrong conclusion.

That’s why modeling is a process of constant questioning:
- Does this equation reflect what I know about the real world?
- Are the units consistent?
- Does the output feel right?
- What happens if I change this assumption?

Great modeling happens at the intersection of disciplines — in the gray space between theory and experience. It's where Python meets physics. Where calculus meets context. Where "how" meets "why."

So whether you're working with runoff or revenue, remember:
- ✅ Don't just trust the math — test your assumptions.
- ✅ Don’t stop at “it runs” — ask does it make sense?

That’s where the real insight begins.

Stay curious, and keep your pencils sharp and your Python clean.

_Thanks for reading!_
